In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
## Set-up Environment
import os
import sys

# Setup: Get notebook directory and set project root
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))
os.chdir(project_root)

# Add project root to Python path for absolute imports like 'env.trading_env'
if project_root not in sys.path:
    sys.path.append(project_root)

# Print confirmation
print("📂 Notebook directory:", notebook_dir)
print("✅ Project root set to:", project_root)

# Change directory to project folder (optional)
%cd /workspaces/Strategy_PPO_Bots


In [ ]:
## Check to ensure in the right directory
! pwd

In [ ]:
## Install dependencies
! pip install -r requirements.txt

In [ ]:
## Run integration test script
#! pytest tests/integration/test_integration.py --maxfail=1 --disable-warnings -q

In [ ]:
## Generate Training Data
! python data_gen/generate_data.py

In [ ]:
## ─── Plot all 1day CSVs in data/ ─────────────────────────────────────────
import glob

import matplotlib.pyplot as plt
import pandas as pd

# 1. Grab every CSV matching *_1day.csv
csv_files = glob.glob("data/*_1day.csv")
if not csv_files:
    raise FileNotFoundError("No files found: data/*_1day.csv")

# 2. Loop and plot
for file_path in sorted(csv_files):
    # Load and parse time
    df = pd.read_csv(file_path, parse_dates=[0])
    df.set_index(df.columns[0], inplace=True)

    # Extract ticker from filename
    ticker = file_path.split("/")[-1].split("_")[0]

    # Create the figure
    plt.figure(figsize=(10, 4))
    plt.plot(df.index, df["close"], linewidth=1)
    plt.title(f"{ticker} 1-Day Close Price")
    plt.xlabel("Time")
    plt.ylabel("Close Price")
    plt.tight_layout()
    plt.show()

In [ ]:
import os

import pandas as pd

# Define the path to your data file
data_file_path = "data/NVDA_1min.parquet"  # Adjust this path if your data is not in a 'data' folder relative to your notebook

# Check if the file exists before attempting to read it
if os.path.exists(data_file_path):
    # Load the parquet file into a pandas DataFrame
    df_check = pd.read_parquet(data_file_path)

    # Print the first few rows to see the data structure
    print("--- First 5 rows of NVDA_1min.parquet ---")
    print(df_check.head())

    # Print all column names to check for 'close' (case-sensitive)
    print("\n--- Column names in NVDA_1min.parquet ---")
    print(df_check.columns.tolist())

    # Check specifically if 'close' column exists
    if "close" in df_check.columns:
        print("\n✅ 'close' column found!")
    elif "Close" in df_check.columns:
        print("\n⚠️ 'Close' (capital C) column found instead of 'close' (lowercase c).")
    else:
        print("\n❌ 'close' column not found in any capitalization.")

else:
    print(
        f"Error: Data file not found at {data_file_path}. Please ensure generate_data.py has been run successfully."
    )

In [ ]:
## Run Integration Test for PPO
# %run train/train_ppo.py --test


In [ ]:
# import torch

# print("CUDA Available:", torch.cuda.is_available())
# print("Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


In [ ]:
# Run PPO training using files in `data/` directory
%run train/train_ppo.py

In [ ]:
#! ls ../debug_rewards.log
